<a href="https://colab.research.google.com/github/RadouaneElarfaoui/anime3rb_scraper/blob/master/anime3rb_scraper_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anime3rb Scraper - Google Colab Notebook

Ce notebook vous permet d'exécuter le scraper Anime3rb directement dans Google Colab. Il installera les dépendances nécessaires et exécutera des exemples d'utilisation.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_GITHUB_USERNAME/YOUR_REPO_NAME/blob/main/anime3rb_scraper.ipynb)

**Note:** Le site anime3rb.com utilise Cloudflare, ce qui peut entraîner des problèmes de scraping. Ce script tente de contourner cette protection en utilisant Playwright, mais le succès n'est pas garanti.

In [ ]:
# 1. Installation des dépendances
!pip install httpx beautifulsoup4 selectolax python-dotenv playwright
!playwright install

In [ ]:
# 2. Téléchargement du module anime3rb_client.py
# Si vous avez ce notebook dans un dépôt GitHub, vous pouvez le télécharger directement.
# Remplacez YOUR_GITHUB_USERNAME et YOUR_REPO_NAME par les vôtres.
!wget -O anime3rb_client.py https://raw.githubusercontent.com/YOUR_GITHUB_USERNAME/YOUR_REPO_NAME/main/anime3rb_client.py

# Alternative: Si vous exécutez ce notebook localement et que le fichier est déjà présent:
# from google.colab import files
# uploaded = files.upload() # Uploader anime3rb_client.py manuellement

In [ ]:
# 3. Exécution des exemples d'utilisation
import asyncio
from anime3rb_client import Anime3rbClient, Anime3rbError, NotFoundError, Pagination, Anime, Episode, EpisodeLink

async def main():
    async with Anime3rbClient() as client:
        print("\n--- Testing list_anime ---")
        try:
            pagination = await client.list_anime(page=1)
            print(f"Found {len(pagination.items)} animes on page 1.")
            if pagination.items:
                first_anime = pagination.items[0]
                print(f"First anime: {first_anime.title} ({first_anime.url})")

                print(f"\n--- Testing get_anime for {first_anime.title} ---")
                full_anime = await client.get_anime(first_anime.url)
                print(f"Full anime details for {full_anime.title}:")
                print(f"  Synopsis: {full_anime.synopsis[:100]}...")
                print(f"  Genres: {', '.join(full_anime.genres)}")
                print(f"  Status: {full_anime.status}")
                print(f"  Rating: {full_anime.rating}")
                print(f"  Year: {full_anime.year}")

                print(f"\n--- Testing list_episodes for {full_anime.title} ---")
                episodes = await client.list_episodes(full_anime.slug)
                print(f"Found {len(episodes)} episodes for {full_anime.title}.")
                if episodes:
                    first_episode = episodes[0]
                    print(f"First episode: {first_episode.title} ({first_episode.url})")

                    print(f"\n--- Testing get_episode_links for {first_episode.title} ---")
                    episode_links = await client.get_episode_links(first_episode.url)
                    print(f"Found {len(episode_links)} links for {first_episode.title}.")
                    for link in episode_links:
                        print(f"  Server: {link.server}, Kind: {link.kind}, URL: {link.url}")
                else:
                    print(f"No episodes found for {full_anime.title}.")
            else:
                print("No animes found on page 1.")

        except Anime3rbError as e:
            print(f"An error occurred during anime listing/detail: {e}")

        print("\n--- Testing search (query: 'naruto') ---")
        try:
            search_results = await client.search("naruto")
            print(f"Found {len(search_results.items)} search results for 'naruto'.")
            if search_results.items:
                for anime in search_results.items[:3]: # Print first 3 results
                    print(f"  - {anime.title} ({anime.url})")
        except Anime3rbError as e:
            print(f"An error occurred during search: {e}")

        print("\n--- Testing list_genres ---")
        try:
            genres = await client.list_genres()
            print(f"Found {len(genres)} genres.")
            print(f"Genres: {', '.join(genres[:10])}...") # Print first 10 genres
        except Anime3rbError as e:
            print(f"An error occurred during genre listing: {e}")

asyncio.run(main())